In [2]:
#This code is optimized to generate the BDCM entropy plots for majority dynamics always-stay tie breaking on Erdos-Renyi graph.
#(with small modifications it can run on any graph with different dynamical rules)

import numpy as np
import networkx as nx
import itertools
import random
import copy
import time

#Ising ferromagnet/majority rule, always stay tie breaking
def atr_condition(xi,xj,rho,p,c):
    if xi[p]==np.sign(rho[p+c-1]+xj[p+c-1]): return(1)
    elif (rho[p+c-1]+xj[p+c-1])==0 and xi[p]==xi[p+c-1]: return(1)
    else: return(0)
        
def traj_condition(xi,xj,rho,p,c):
    prod=1
    for t in range(0,p+c-1):
        #if x_i[t+1]==-1*np.sign(np.sum(neighbours[:,t])): continue    #minority dynamics
        if xi[t+1]==np.sign(rho[t]+xj[t]): continue    #majority dynamics
        elif (rho[t]+xj[t])==0 and xi[t+1]==xi[t]: continue #always-stay tie breaking
        #elif np.sum(neighbours[:,t])==0 and x_i[t+1]==-1*x_i[t]: continue #always-change tie breaking
        else: 
            prod=0
            break
    return(prod)

def atr_condition2(xi,rho,p,c):
    if xi[p]==np.sign(rho[p+c-1]): return(1)
    elif (rho[p+c-1])==0 and xi[p]==xi[p+c-1]: return(1)
    else: return(0)
        
def traj_condition2(xi,rho,p,c):
    prod=1
    for t in range(0,p+c-1):
        #if x_i[t+1]==-1*np.sign(np.sum(neighbours[:,t])): continue    #minority dynamics
        if xi[t+1]==np.sign(rho[t]): continue    #majority dynamics
        elif (rho[t])==0 and xi[t+1]==xi[t]: continue #always-stay tie breaking
        #elif np.sum(neighbours[:,t])==0 and x_i[t+1]==-1*x_i[t]: continue #always-change tie breaking
        else: 
            prod=0
            break
    return(prod)

def m_attr_i(x_i,p,c):
    return(np.sum(x[p:])/c)

def attr_fix(x_i,p,c,attr_value):
    if x_i[p+c-1]==attr_value: return(1)
    else: return(0)

def A_i_sums(xi,xj,rho,p,c,attr_value,lmbd_in):
    return(np.exp(-lmbd_in*xi[0])*atr_condition(xi,xj,rho,p,c)*traj_condition(xi,xj,rho,p,c)*attr_fix(xi,p,c,attr_value))

def A_i_sums2(xi,rho,p,c,attr_value,lmbd_in):
    return(np.exp(-lmbd_in*xi[0])*atr_condition2(xi,rho,p,c)*traj_condition2(xi,rho,p,c)*attr_fix(xi,p,c,attr_value))

def onestep_majority(nodes_with_d_positions,degrees_nodes,N_nodes_pos,s):
    sums=np.zeros_like(s)
    for d in degrees_nodes:
        sums[nodes_with_d_positions[d]]=np.sum(s[N_nodes_pos[d]], axis=1) 
    return np.sign(2*sums + s)

#returns endstate of node values
def s_endstate(nodes_with_d_positions,degrees_nodes,N_nodes_pos,s,p,c):
    for k in range(p+c-1):
        s=onestep_majority(nodes_with_d_positions,degrees_nodes,N_nodes_pos,s)
    return(s)

def m(s,n):
    return(np.sum(s)/n)

def normalize(chi):
    sum_axes = tuple(range(1, chi.ndim)) #all axes except the first one (first one is the "2*edges" one)
    return chi/np.sum(chi, axis=sum_axes, keepdims=True)

#here we also take into account the all ones attractor (we dont sum through states ending with -1)
def BDCM_ER(chi,degrees_edges,edges_with_d_positions,N_edges_pos_dm1,A,p,c,attr_value,lmbd_in,damppar,epsilon):
    L = {}
    LL = {}
    chi_local = {}
    chi2 = {}
    for d in degrees_edges:  #simplest we can do is do the RRG HPR for every d (later/if needed we can maybe try to do it so that all degrees stuff are updated at once, but this approach probably needs more memory)
        if d>0:  #edges (j,i) with 0 messages needed for the BDCM update (j is a leaf) need special treatment(we update them only once at the begining, code below) 
        
            #d is now really the number of messages needed for the BDCM update, but in the RRG it was node degree (so we needed d-1 messages for the update) 
            #hence: on the RHS we need to use d+1 (e.g. when defining LL)
        
            #L(number of mes with given d mes needed for the BDCM update (except j), traj. xi, sums traj. rho)
            L[d] = np.zeros(([np.array(edges_with_d_positions[d])[0].size]+[2]*T+[d+1]*T))
            LL[d] = np.zeros(([np.array(edges_with_d_positions[d])[0].size]+[2]*T+[d+1]*T))
            chi_local[d] = chi[N_edges_pos_dm1[d]] #chi_local have all the message values needed for the message ij update in the corresponding ij "row"
            
            #we start(D=1) with just message values and trajectory "sum" of just trajectory values (rho_(D=1)=x_k)
            for xi in itertools.product([1, 0], repeat=T):
                if xi[-1] == attr_value:
                    for xk in itertools.product([1, 0], repeat=T):
                        if xk[-1] == attr_value:
                            LL[d][tuple([slice(None)] + list(xi) + list(xk))] = chi_local[d][tuple([slice(None), 0] + list(xk) + list(xi))] 
            
            for D in range(2,d+1): #we go with D from 1(above), 2, 3,... to d-1 now +1 --see note above (number of neighbours except j)
                L[d] = np.zeros_like(LL[d])
                for xi in itertools.product([1, 0], repeat=T):
                    if xi[-1] == attr_value:
                        for xk in itertools.product([1, 0], repeat=T):
                            if xk[-1] == attr_value:
                                idx_rho_D = tuple([slice(None)]
                                                  + list(xi)
                                                  + [slice(xk[t], D + xk[t]) for t in range(T)]
                                ) #we want to create L values for all messages and xi at once---for all possible rho_D's, rho_D's are stored
                                #in the last array positions and for given D we have rho_D = rho_D-1 + xk, so we only access thsose positions
                                #that start with at least xk values!! hence slice(xk[t] --we go up D-1 values because that is the maximum for rho we can have for given D (all  previous neighbors were 1))
                                # on the RHS we need to match shapes, so we take from 0 to D-1 (there are either zeros, or indeed values that we need from previous D-1)--this is the index idx_rho_Dm1            
                                
                                idx_rho_Dm1 = tuple([slice(None)]
                                                    + list(xi)
                                                    + [slice(None, D) for _ in range(T)]
                                )
                    
                                #above we took message from the zeroth neighbor of chi_local, here D-1 takes first, second...
                                idx_mes = tuple([slice(None), D-1]
                                                + list(xk)
                                                + list(xi)
                                )
                                
                                
                                L[d][idx_rho_D] += LL[d][idx_rho_Dm1]*chi_local[d][idx_mes][tuple([slice(None)] + [np.newaxis] * T)]         #it creates (slice(None), None, None...) so that we can math the shapes of LL and chi_local[] and do the element wise multiplication
                    
                LL[d]=L[d].copy()
            
            #final HPr update
            chi2[d]=np.zeros_like(chi[edges_with_d_positions[d]])
            for xi in itertools.product([1, 0], repeat=T):
                if xi[-1] == attr_value:
                    for xj in itertools.product([1, 0], repeat=T):
                        chi2[d][tuple([slice(None)] + list(xi) + list(xj))] += np.sum(np.exp(-lmbd_in*(2*xi[0]-1))*A[d][tuple(list(xi) + list(xj))][tuple([np.newaxis] + [slice(None)])] * LL[d][tuple([slice(None)] + list(xi))], axis=tuple(range(1,T+1)))
                
            
            #normalizations of mes + dampening
            chi2[d] = np.maximum(chi2[d],epsilon)
            chi[edges_with_d_positions[d]] = damppar*normalize(chi2[d]) + (1-damppar)*chi[edges_with_d_positions[d]]

    return chi

def Zij(chi,num_edg,epsilon):
    #we sum products of chi^ij_xixj*chi^ji_xjxi with the right pairs (xi, xj are really the same in both xixj and xjxi) and store this for each edge in Zijs
    Zijs=np.zeros(num_edg)
    for xi in itertools.product([1, 0], repeat=T):
        if xi[-1] == attr_value:
            for xj in itertools.product([1, 0], repeat=T):
                if xj[-1] == attr_value:
                    Zijs += chi[tuple([slice(0,num_edg)] + list(xi) + list(xj))]*chi[tuple([slice(num_edg,2*num_edg)] + list(xj) + list(xi))]

    return np.maximum(Zijs,epsilon)

def Zi_ER(chi,degrees_nodes,nodes_with_d_positions,N_edges_pos_full,Ai,p,c,N,attr_value,lmbd_in):    
    L = {}
    LL = {}
    chi_neib = {}
    Zis = {}
    for d in degrees_nodes:  #simplest we can do is do the RRG HPR for every d (later/if needed we can maybe try to do it so that all degrees stuff are updated at once, but this approach probably wastes memory)
        if d>0:  #isolated nodes can be ignored
        
            #d is now really the number of messages needed for the BDCM update, but in the RRG it was node degree (so we needed d-1 messages for the update) 
            #hence: on the RHS we need to use d+1 (e.g. when defining LL)
        
            #L(number of mes with given d mes needed for the BDCM update (except j), traj. xi, sums traj. rho)
            L[d] = np.zeros(([np.array(nodes_with_d_positions[d]).size]+[2]*T+[d+1]*T))
            LL[d] = np.zeros(([np.array(nodes_with_d_positions[d]).size]+[2]*T+[d+1]*T))
            chi_neib[d] = chi[N_edges_pos_full[d]] #chi_neib have all the message values needed for the message Zi -all neighboring messages
            
            #we start(D=1) with just message values and trajectory "sum" of just trajectory values (rho_(D=1)=x_k)
            for xi in itertools.product([1, 0], repeat=T):
                if xi[-1] == attr_value:
                    for xk in itertools.product([1, 0], repeat=T):
                        if xk[-1] == attr_value:
                            LL[d][tuple([slice(None)] + list(xi) + list(xk))] = chi_neib[d][tuple([slice(None), 0] + list(xk) + list(xi))] 
                
            for D in range(2,d+1): #we go with D from 1(above), 2, 3,... up to d (all neib)
                L[d] = np.zeros_like(LL[d])
                for xi in itertools.product([1, 0], repeat=T):
                    if xi[-1] == attr_value:
                        for xk in itertools.product([1, 0], repeat=T):
                            if xk[-1] == attr_value:
                                idx_rho_D = tuple([slice(None)]
                                                  + list(xi)
                                                  + [slice(xk[t], D + xk[t]) for t in range(T)]
                                ) #we want to create L values for all messages and xi at once---for all possible rho_D's, rho_D's are stored
                                #in the last array positions and for given D we have rho_D = rho_D-1 + xk, so we only access thsose positions
                                #that start with at least xk values!! hence slice(xk[t] --we go up D-1 values because that is the maximum for rho we can have for given D (all  previous neighbors were 1))
                                # on the RHS we need to match shapes, so we take from 0 to D-1 (there are either zeros, or indeed values that we need from previous D-1)--this is the index idx_rho_Dm1            
                                
                                idx_rho_Dm1 = tuple([slice(None)]
                                                    + list(xi)
                                                    + [slice(None, D) for _ in range(T)]
                                )
                    
                                #above we took message from the zeroth neighbor of chi_local, here D-1 takes first, second...
                                idx_mes = tuple([slice(None), D-1]
                                                + list(xk)
                                                + list(xi)
                                )
                                
                                
                                L[d][idx_rho_D] += LL[d][idx_rho_Dm1]*chi_neib[d][idx_mes][tuple([slice(None)] + [np.newaxis] * T)]         #it creates (slice(None), None, None...) so that we can math the shapes of LL and chi_local[] and do the element wise multiplication
                    
                LL[d]=L[d].copy()
            
            #finally
            Zis[d]=np.zeros(np.array(nodes_with_d_positions[d]).size)
            for xi in itertools.product([1, 0], repeat=T):
                if xi[-1] == attr_value:
                    Zis[d] += np.exp(-lmbd_in*(2*xi[0]-1))*np.sum(Ai[d][tuple(list(xi))][tuple([np.newaxis] + [slice(None)])] * LL[d][tuple([slice(None)] + list(xi))],axis=tuple(range(1,T+1)))
        
    
    Zis_fin =np.zeros(N)
    for d in degrees_nodes:
        if d>0:
            Zis_fin[nodes_with_d_positions[d]] = Zis[d]

    return np.maximum(Zis_fin,epsilon)

def GENERAL_ERgraph_and_auxialiaryarrays_generation(n,prob,p,c,T,attr_value):
    #RANDOM GRAPH GENERATION (General sparse ER) -----------------------------------------------------------------------------------------------
    G_first = nx.fast_gnp_random_graph(n, prob)

    #it can happen that we have isolated nodes (nodes without neighbors)
    isolated_nodes = list(nx.isolates(G_first))
    number_iso = len(isolated_nodes) #number of isolated nodes, they just add -lambda_init*number_iso/n to the free entropy densitiy in our case
    avg_deg = sum(dict(G_first.degree()).values()) / G_first.number_of_nodes()
    
    #we remove them (we run BDCM on the parts of G which are not isolated (contibution to phi is done as we stated above))
    G = G_first.copy()
    G.remove_nodes_from(isolated_nodes)
    
    G = nx.convert_node_labels_to_integers(G)
    N_G_without_isolated = G.number_of_nodes()
    num_edg=G.number_of_edges()
    adj_matrix = nx.to_numpy_array(G)
    
    #AUXILIARY ARRAYS ---------------------------------------------------------------------------------------------------------------------
    degrees_all = [d for _, d in G.degree()] #for degree i (ith row) we have its degree
    degrees_nodes=np.unique(degrees_all) 
    
    #N_nodes is  a dictionary of neighboring nodes, row i containis neighboring nodes of i
    N_nodes = {node: list(G.neighbors(node)) for node in G.nodes()}
    
    edge_dict = {} #edge_dict[(i,j)] returns an index/order in G.edges of the edge (i,j), if (i,j) is a reversed edge (still an edge but not in G.edges) it returns index of (j,i)+num_edges
    edges = np.zeros((2*num_edg,2)) #we will have the edges of a graph in an array including the reversed ones in the second half
    edges_degree = np.zeros(2*num_edg) #contains number of neighboring edges of edge (i,j) (edges that end in i, except (j,i), second half for reversed edges 
    for idx, edge in enumerate(G.edges):        
        edge_dict[edge] =idx
        edge_dict[edge[::-1]] = (idx+num_edg)
    
        edges[idx]=edge
        edges[idx+num_edg]=edge[::-1]
    
        edges_degree[idx] = degrees_all[edge[0]] -1
        edges_degree[idx+num_edg] = degrees_all[edge[1]] -1
    edges=edges.astype(int)
    edges_degree=edges_degree.astype(int)
    
    degrees_edges = np.unique(edges_degree)
    
    #now we create all the auxiliary arrays (like neighboring edges positions and so on), but for node groups with the same degree
    A = {}
    Ai = {}
    N_edges_pos_dm1 = {}
    N_edges_pos_full = {}
    N_edges_pos_full_marginals = {}
    N_nodes_pos = {}
    edges_with_d_positions = {}
    nodes_with_d_positions = {}
    
    for d in degrees_edges:
        edges_with_d_positions[d] = np.where(edges_degree==d) #positions of edges (i,j) that have d neighboring edges (node i has d+1 neighbors)
        
        #now we create an array of neigboring edges POSITIONS to an edge (i,j) as they go in G.edges, but in a way where we have edges (k1,i), (k2,i)... where k's are from the neib of i except j! 
        #we need to know these edges, as they correspond to the messages needed for the BDCM update
        N_edges_pos_dm1[d] = np.array(
                [[edge_dict[(k, i)] for k in N_nodes[i] if k != j] for i, j in edges[edges_with_d_positions[d]]]
        )
    
        #for the given rule we construct A matrix with factor node A_i values (we could speed up the interations here, but we only go through them once for lmbd_in=0 (!!)
        if d>0:
            A[d]=np.zeros([2]*T + [2]*T + [d+1]*T)   #A(xi,xj,rho_d), now d is really number of neib edges needed for the BDCM update (d-1 in case of RRG)
            for xi in itertools.product([1, 0], repeat=T):
                for xj in itertools.product([1, 0], repeat=T):
                    for rho in itertools.product(np.arange(d+1), repeat=T):
                        A[d][tuple(list(xi) + list(xj) + list(rho))] = A_i_sums(2*np.array(xi)-1,2*np.array(xj)-1,2*np.array(rho)-d,p,c,attr_value,lmbd_in=0)
    
        
    for d in degrees_nodes:
        nodes_with_d_positions[d] = np.where(np.array(degrees_all)==d)[0]
        #now including the j and only for nodes i (rows correspond to nodes not edges) --we use this when computing Zi and majority dynamics step
        N_edges_pos_full[d] = np.array(
                [[edge_dict[(k, i)] for k in N_nodes[i]] for i in nodes_with_d_positions[d]]
        )
    
        N_edges_pos_full_marginals[d] = np.array(
                [[edge_dict[(i, k)] for k in N_nodes[i]] for i in nodes_with_d_positions[d]]
        )
    
        N_nodes_pos[d] = np.array(
                [[k  for k in N_nodes[i]] for i in nodes_with_d_positions[d]]
        )
    
        #also for Zi construction
        Ai[d]=np.zeros([2]*T + [d+1]*T)   #A(xi,rho_d)
        for xi in itertools.product([1, 0], repeat=T):
            for rho in itertools.product(np.arange(d+1), repeat=T):
                Ai[d][tuple(list(xi) + list(rho))] = A_i_sums2(2*np.array(xi)-1,2*np.array(rho)-d,p,c,attr_value,lmbd_in=0)

    return avg_deg, N_G_without_isolated, number_iso, num_edg, adj_matrix, degrees_all, degrees_nodes, N_nodes, A, Ai, N_edges_pos_dm1, N_edges_pos_full, N_edges_pos_full_marginals, N_nodes_pos, edges_with_d_positions, nodes_with_d_positions, degrees_edges, edges 


def phi_BP_GENERAL_ER(chi,num_edg,degrees_nodes,nodes_with_d_positions,N_edges_pos_full,Ai,N_G_without_isolated,n,p,c,attr_value,lmbd_in,epsilon, number_iso):
    Zis=Zi_ER(chi,degrees_nodes,nodes_with_d_positions,N_edges_pos_full,Ai,p,c,N_G_without_isolated,attr_value,lmbd_in)
    Zijs=Zij(chi,num_edg,epsilon)
    
    return (np.sum(np.log(Zis)) - np.sum(np.log(Zijs)) - lmbd_in*number_iso)/n


def avg_m_init_GENERAL_ER(chi,num_edg,degrees_all,edges,n,epsilon,number_iso):
    sum=np.zeros(num_edg)
    Zijs=np.zeros(num_edg)
    for xi in itertools.product([1, 0], repeat=T):
        if xi[-1] == attr_value:
            for xj in itertools.product([1, 0], repeat=T):
                if xj[-1] == attr_value:
                    sum += ((2*xi[0]-1)/np.array(degrees_all)[edges[0:num_edg,0]] + (2*xj[0]-1)/np.array(degrees_all)[edges[0:num_edg,1]])*(chi[tuple([slice(0,num_edg)] + list(xi) + list(xj))]*chi[tuple([slice(num_edg,2*num_edg)] + list(xj) + list(xi))])
                    Zijs += chi[tuple([slice(0,num_edg)] + list(xi) + list(xj))]*chi[tuple([slice(num_edg,2*num_edg)] + list(xj) + list(xi))]
            
    Zijs=np.maximum(Zijs,epsilon)
    sum=sum/Zijs
                    
    return (np.sum(sum) + number_iso)/n

def BDCM_entropy_procedure_GENERAL_ER(chi, lambdas, T_max,n_saves,saving_time,start):
    count=0
    counts= 0

    ent = np.zeros(lambdas.size)
    m_init = np.zeros(lambdas.size)
    ent1 = np.zeros(lambdas.size)
    
    #we go through the range of lambda values (starting at lambda=0 and fixed point from previous lambda value as initial value for mes for next lambda value)
    for lmbd_in in lambdas:
        if degrees_edges[0]==0:
            #first for new lambda, we update leaves messages
            d=0 #these are the edges (j,i) where node j is a leaf, hence here the BDCM update is simply just normalized factor node A_j
            #so the BDCM update is: 
            chi_aux=np.zeros_like(chi[edges_with_d_positions[d]])
            zero_sum=np.zeros(T)
            for xi in itertools.product([1, 0], repeat=T):
                if xi[-1] == attr_value:
                    for xj in itertools.product([1, 0], repeat=T):
                        chi_aux[tuple([slice(None)] + list(xi) + list(xj))] = A_i_sums(2*np.array(xi)-1,2*np.array(xj)-1,zero_sum,p,c,attr_value,lmbd_in)
                
            #normalizations of mes (dampening is not needed, on leaves we have this exact values unchanging)
            chi_aux=normalize(chi_aux)
            chi[edges_with_d_positions[d]] = chi_aux
    
        
        delta=1
        t=0  
        while(delta>eps):
            chi_copy=chi.copy()
        
            chi= BDCM_ER(chi,degrees_edges,edges_with_d_positions,N_edges_pos_dm1,A,p,c,attr_value,lmbd_in,damppar,epsilon)
            
            delta=np.abs(chi-chi_copy).max()
            t+=1
            if t>=T_max:
                delta=0
                counts = lmbd_in
                
        print("lambda=",lmbd_in," t=",t," eps-delta=",eps-delta)
        ent[count] = phi_BP_GENERAL_ER(chi,num_edg,degrees_nodes,nodes_with_d_positions,N_edges_pos_full,Ai,N_G_without_isolated,n,p,c,attr_value,lmbd_in,epsilon, number_iso)     
        m_init[count] = avg_m_init_GENERAL_ER(chi,num_edg,degrees_all,edges,n,epsilon,number_iso)
        ent1[count] = ent[count] + lmbd_in*m_init[count]
        print("m_init:",m_init[count], "ent: ", ent1[count])

        '''
        cur_time=time.time()
        if ((cur_time-start)>saving_time) and n_saves<1: 
            print(cur_time-start,"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            #np.savez("ER_p1_auto_TEST.npz", m_init=m_init, ent1=ent1, ent=ent,nodes_numbers=nodes_numbers, mean_degrees=mean_degrees, max_degrees=max_degrees, deg=deg, prob=prob,mean_degrees_total=mean_degrees_total,nodes_isolated=nodes_isolated, T_max=T_max,num_rep=num_rep)
            n_saves+=1
        '''
        if ent1[count]< -0.05: break
        if counts>0: break #when counts is non-empty I break the loop, this means that we have hit the lambda for which the mes didnt converge under T_max  steps (and we store in counts this value)
            
        count+=1


    return m_init, ent1, ent, counts


#PARAMETERS -----------------------------------------------------------------------------------------------
n=1000 #number of nodes 
#Erdos-Renyi (ER) graph specification:
#let deg be the mean node degree: deg=(n-1)*p
deg=np.linspace(1,2,3)
#deg=np.array([4])
prob = deg/(n-1)

num_rep=3

# (p+c) backtracking attractors
p=1
c=1
T=p+c

eps=1e-6 #precision of the converged BDCM messages
damppar=0.1
attr_value=1
epsilon=0 #lowest possible Z_i or Z_ij value and chi mes

saving_time=30
n_saves=0

T_max=1300

a=12
dl=0.1
lambdas=np.linspace(0,a,int(a/dl+1))

ent = np.zeros((deg.size,num_rep,lambdas.size))
m_init =  np.zeros((deg.size,num_rep,lambdas.size))
ent1 =  np.zeros((deg.size,num_rep,lambdas.size))

nodes_numbers = np.zeros((deg.size,num_rep))
mean_degrees = np.zeros((deg.size,num_rep))
max_degrees = np.zeros((deg.size,num_rep))
nodes_isolated = np.zeros((deg.size,num_rep))
mean_degrees_total = np.zeros((deg.size,num_rep))

start=time.time()

for idx, _ in enumerate(prob):
    for n_rep in range(num_rep):
        avg_deg, N_G_without_isolated, number_iso, num_edg, adj_matrix, degrees_all, degrees_nodes, N_nodes, A, Ai, N_edges_pos_dm1, N_edges_pos_full, N_edges_pos_full_marginals, N_nodes_pos, edges_with_d_positions, nodes_with_d_positions, degrees_edges, edges = GENERAL_ERgraph_and_auxialiaryarrays_generation(n,prob[idx],p,c,T,attr_value)
        
        nodes_isolated[idx][n_rep] = number_iso
        mean_degrees[idx][n_rep] = np.mean(degrees_all)
        mean_degrees_total[idx][n_rep] = avg_deg
        max_degrees[idx][n_rep] = degrees_nodes.max()
        print()
        print("deg:",deg[idx], "isolated nodes:",number_iso, "avg_degree_total:",avg_deg)
        print()
        
        #messages-----------------------------------------------
        chi=np.random.random(([2*num_edg] + [2]*T + [2]*T))
        chi=normalize(chi)
        
        #m_init, ent1, ent, counts = BDCM_entropy_procedure(chi, lambdas, T_max)
        m_init[idx][n_rep], ent1[idx][n_rep], ent[idx][n_rep], counts = BDCM_entropy_procedure_GENERAL_ER(chi, lambdas, T_max,n_saves,saving_time,start)

#np.savez("ER_p1.npz", m_init=m_init, ent1=ent1, ent=ent,nodes_numbers=nodes_numbers, mean_degrees=mean_degrees, max_degrees=max_degrees, deg=deg, prob=prob,mean_degrees_total=mean_degrees_total,nodes_isolated=nodes_isolated, T_max=T_max,num_rep=num_rep)



deg: 1.0 isolated nodes: 370 avg_degree_total: 0.97

lambda= 0.0  t= 159  eps-delta= 2.224418177332228e-08
m_init: 0.7859766580538275 ent:  0.1720699495590459
lambda= 0.1  t= 130  eps-delta= 2.2140530232519043e-09
m_init: 0.7699358367558866 ent:  0.17127259171924963
lambda= 0.2  t= 135  eps-delta= 5.178161091479079e-08
m_init: 0.7545492129205356 ent:  0.16897079877838897
lambda= 0.30000000000000004  t= 138  eps-delta= 3.3195433336565375e-09
m_init: 0.7399806499309954 ent:  0.16533606458353123
lambda= 0.4  t= 142  eps-delta= 5.081022252839591e-08
m_init: 0.7263552613663471 ent:  0.1605754636000715
lambda= 0.5  t= 145  eps-delta= 4.151309743083327e-08
m_init: 0.7137593656167142 ent:  0.15491615729839237
lambda= 0.6000000000000001  t= 149  eps-delta= 5.117577753133058e-08
m_init: 0.7022428278329915 ent:  0.14859118078564132
lambda= 0.7000000000000001  t= 152  eps-delta= 1.3671754921617162e-08
m_init: 0.6918229572378949 ent:  0.14182740343380668
lambda= 0.8  t= 156  eps-delta= 9.103118646

KeyboardInterrupt: 